# Get services from remote computer

Open an powershell remote session to the computer and run the code inside of the ScriptBlock. Press **Run all** to execute all tests.

### Define the remote computer to connect to. It can be a single computer or a list of computers.

In [ ]:
$remote = 'TINU0034545.domain.com'
$creds  = $null

# ScriptBlock
$scriptblock = {
    param(
        [Parameter(Mandatory=$false)]
        [String[]] $params = @('vmtools', 'ccmexec', 'puppet', 'w32time', 'winrm', 'termservice', 'winmgmt', 'wuauserv', 'bits')
    )

    function Get-ManagedServiceStatus{
        [CmdletBinding()]
        param(  
            [Parameter(Mandatory=$false)]
            [String[]] $Service
        )
        foreach($item in $Service){
            #$ret = Get-Service -Name $item
            $ret = Get-CimInstance -ClassName win32_service -Filter "name='$item'" -ErrorAction SilentlyContinue
            [PSCustomObject]@{
                ComputerName       = [System.Net.Dns]::GetHostByName($env:computername).HostName
                LastBootupTime     = (Get-CimInstance Win32_OperatingSystem).LastBootupTime
                TemplateVersion    = Get-ItemPropertyValue HKLM:\SOFTWARE\Swisscom\ImageInfo -Name TemplateVersion
                ServiceName        = $ret.Name
                ServiceDisplayName = $ret.DisplayName
                ServiceStartType   = $ret.StartMode
                ServiceStatus      = $ret.State
                ServicePath        = $ret.PathName
            }
        }
    }
    Get-ManagedServiceStatus -Service $params
}

### Create the object to return from all remote computers

The following services will be checked 'vmtools', 'ccmexec', 'puppet', 'w32time', 'winrm', 'termservice', 'winmgmt', 'wuauserv', 'bits'

- ComputerName
- LastBootupTime
- TemplateVersion
- ServiceName
- ServiceDisplayName
- ServiceStartType
- ServiceStatus

### Invoke PowerShell Remote Script

In [ ]:
# Invoke PowerShell Remote Script
"[INFO] Start $(Get-Date -f 'yyy-MM-dd HH:mm:ss.fff')"; $StartDate = Get-Date
if([String]::IsNullOrEmpty($creds)){
    $creds = Get-Credential  -UserName "$(($env:USERNAME).ToLower())@swisscom.com" -Message "Enter your credentails for remote systems"
}
foreach($item in $remote){
    $rsession = New-PSSession -ComputerName $item -Credential $creds -ErrorAction Stop
    if($rsession){
        $ret = Invoke-Command -Session $rsession -ScriptBlock $scriptblock -ArgumentList $null
        $ret | Format-Table -GroupBy ComputerName -property @('LastBootupTime','ServiceName','ServiceStatus','ServiceStartType','ServicePath')
        Remove-PSSession -Session $rsession
    }
}
"[INFO] Stop $(Get-Date -f 'yyy-MM-dd HH:mm:ss.fff')"
$TimeSpan = New-TimeSpan -Start $StartDate -End (Get-Date) | Select -ExpandProperty TotalMilliseconds
"[{0:N0}ms] -> finish" -f $TimeSpan